Import required libraries

In [5]:
import os
import sys

import time
import warnings
import datetime
import random
import pickle
import numpy as np
import pandas as pd


import torch
from torch import nn
import torch.multiprocessing
import torch.distributed as dist
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# import torchvision
# from torchvision import transforms

from transformers import AutoModel, AutoModelWithLMHead, AutoModelForCausalLM, pipeline
#from datasets import load_dataset, load_from_disk
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from accelerate import Accelerator


Set required env variables

In [6]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["TORCH_USE_CUDA_DSA"] = "1"
# os.environ["RANK"] = "0"
# os.environ["LOCAL_RANK"] = "0"
# os.environ["WORLD_SIZE"] = "1"
# os.environ["MASTER_PORT"] = "29580"
# os.environ["MASTER_ADDR"] = "127.0.0.1"

In [7]:
def build_prompt(passage):

    sys_message = "You are a writing AI assistant. rewrite the provided text using official language."
    user_message = "Here is the passage: \n {}. ".format(passage)

    message = [
        { "role" : "system", "content" : sys_message},
        { "role" : "user", "content" : user_message}
        ]

    return message

In [8]:
true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("Fake.csv")

In [9]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [10]:
true_sample = true_df['text'][0]
# len(sample_t)
# print(sample_t)
true_df['subject'].unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [11]:
fake_sample = fake_df['text'][0]
fake_sample

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [12]:
fake_df['subject'].unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [15]:
#model_id = "meta-llama/Llama-3.2-3B"
model_id = "meta-llama/Llama-3.2-3B-Instruct" # better adherence to instructions

pipe = pipeline(
    task="text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype" : torch.bfloat16}, 
    device_map="auto",
    max_new_tokens=512,
    return_full_text=False
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [16]:
msg = build_prompt(fake_sample)
pipe(msg)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 